# Tensorflowを用いたword2vecの実装
このハンズオンでは，Tensorflowを用いて単語の埋め込み表現であるword2vecを実装し，学習する．
## 目次
1. [word2vecの説明](#説明)
- [実装](#実装)
- [使用例](#使用例)

<a name='説明'></a>
## 1. word2vecの説明
## 分散表現
単語のベクトル表現において，最も単純な方法の１つに単語の辞書を作りonehotのベクトルで表現することができる.<br>
しかし全ての単語を独立に扱うと'new'と'novel'は意味が近いといった情報を考慮できない.<br>
これに対して分散表現とは，単語間の関係性を埋め込み，単語を低次元のベクトルで表現したものである.

<img src="figure/onehot_dist.png", width=400>

## word2vecとは？
単語の分散表現の一種. 類似した文脈で登場する単語は似た意味を持つという仮説に基づく.<br>
[参考論文]
- Efficient Estimation of Word Representations in Vector Space [Mikolov et al., ICLR workshop, 2013]<br>
https://openreview.net/forum?id=idpCdOWtqXd60
- Distributed Representations of Words and Phrases and their Compositionality [Mikolov et al., NIPS, 2013]<br>
https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality


## word2vecの学習方法の概要
2つのアプローチが提案されている
- Continuous Bag-of-Words (CBOW)<br>
周辺の単語から単語を予測できるように単語の埋め込みを学習<br>
- Skip-Gram<br>
周辺の単語を予測できるように単語の埋め込みを学習<br>

<img src="figure/skip_cbow.png", width=500>

今回はSkip-Gramを実装する.
モデルは2層の線形変換によって学習する．<br>
単語数を$n$，求めるベクトルの次元を$m$とすると，Skip-Gramは以下の図のようになる．<br>
ネットワークの入力の単語に対して，周辺単語の出現確率が出力されることを目的として学習する．<br>
これによって同じような文脈で登場しやすい単語同士のベクトルが近くなるに学習される
<img src="figure/neuralnet.png", width=300>


<a name='実装'></a>
## 2. 実装
## 2.1 下準備
### Tensorflowのインストール

In [ ]:
!sudo pip install tensorflow-gpu==1.2

### ライブラリのインポート

In [ ]:
from __future__ import print_function

import os
import random
import zipfile
import math
import shutil
import glob
from collections import Counter
from six.moves.urllib.request import urlretrieve

import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from test_word2vec import ans_make_pair

os.environ["CUDA_VISIBLE_DEVICES"]='0'

### データセットの読み込み
データセットは今回はptbデータセットというものを用いる.<br>
https://github.com/tomsercu/lstm/tree/master/data

In [ ]:
file_list = ['data/ptb.train.txt', 'data/ptb.valid.txt', 'data/ptb.test.txt']
all_lines = []
for file in file_list:
    f=open(file)
    all_lines.extend(f.readlines())
    f.close()
    
data =[]
for line in all_lines:
    line = line.strip('\n').lower().split()
    data.append(line)
print('lines : ', len(data))

## 2.2 前処理
1. 頻度の少ない単語を無視して，単語の辞書を作る. 例：{'apple': 0, 'orange': 1, 'banana': 2}<br>
<理由>出現頻度の少なすぎる単語はあまり学習できずノイズになる場合が多い.また、単語の辞書が大きくなりすぎると計算量やメモリが膨大になるので頻度に閾値を設ける.<br><br>
1. 辞書を用いて文章中の単語をidに変換する
1. Skip-Gram用にデータを整形

### 2.2.1 頻度が少ない単語は無視して辞書を作成
※ (今回用いるptbは出現回数5回未満の単語はunknownに置き換わっているなど、前処理がすでに行われている.)

In [ ]:
min_freq = 5

words = []
for sentence in data:
    words.extend(sentence)
word_cnt = Counter(words)

word2id = {'<unk>':0}
id2word = {0:'<unk>'}

for word, cnt in word_cnt.most_common():
    if cnt<min_freq:
        break
    if word !='<unk>':
        word2id[word] = len(word2id)
        id2word[len(id2word)] = word
    
print('vocabulary number is : ', len(word2id))

### 2.2.2 辞書を用いて文章中の単語をidに変換

In [ ]:
converted_data = []
for sentence in data:
    id_sentence = []
    for word in sentence:
        if word in word2id:
            id_sentence.append(word2id[word])
        else:
            id_sentence.append(word2id['<unk>'])
    converted_data.append(id_sentence)
    
print('<ex>')
print('----original sentence---- : ') 
print(data[2])
print('---> id sentence : ')
print (converted_data[2])

### 2.2.3 ターゲットの単語と予測したい周辺単語のペアを作る
- 何個隣まで予測するか(window幅について)<br>
window幅を大きく取ればより広い文脈を取ることができるが，window幅を大きくしすぎると関係のない文脈まで考慮してしまうというトレードオフがある
<img src="figure/window.png", width=200>

今回は予めターゲットの単語と予測したい周辺の単語をペアとしてまとめ，訓練時にランダムで読みだすようにする.<br>
つまり，'I want to eat an orange'はwindow幅が2の時，以下のようなペアを作る.<br>
(I, want), (I, to), (want, I), (want, to), (want, eat), ... , (an, to), (an, eat), (an, orange)

### <font color="red">TODO</font> make_pair関数を埋めよう
- 入力
    - 文章のリスト, window幅
- 出力
    - X_train ... ターゲットの単語のリスト
    - y_train ... 予測したい単語のリスト
- 例        
入力が['I', 'want', 'to', 'eat', 'an', 'orange'], window=2の時，以下のような出力とする.<br>
X_train = ['I', 'I', 'want', 'want', 'want', 'to', 'to', 'to', 'to', 'eat', 'eat', 'eat', 'eat', 'an', 'an', 'an', 'orange', 'orange']<br>
y_train = ['want', 'to', 'I', 'to', 'eat', 'I', 'want', 'eat', 'an', 'want', 'to', 'an', 'orange', 'to', 'eat', 'orange', 'eat', 'an']<br>

In [ ]:
def make_pair(sentence, window_size):
    X_train = []
    y_train = []
    ## TODO
    for target_index in range(len(sentence)):
        for context_index in range(max(target_index - window_size, 0), min(target_index + window_size+1, len(sentence))): 
            if target_index != context_index:
                X_train.append(sentence[target_index])
                y_train.append(sentence[context_index])
    ## TODO
    return X_train, y_train

sentence = ['I', 'want', 'to', 'eat', 'an', 'orange']
window = 2

x,y = make_pair(sentence, window)
print(x)
print(y)
assert ans_make_pair(sentence, window)==make_pair(sentence, window)

In [ ]:
skip_window = 2 # 何個隣までを予測するか
X_train = []
y_train = []

for sentence in converted_data:
    x, y = make_pair(sentence, skip_window)
    X_train.extend(x)
    y_train.extend(y)
                
X_train = np.array(X_train, dtype=np.int32)
y_train = np.array(y_train, dtype=np.int32)
print('train data : ', len(X_train))
print ('<ex> x : ', X_train[0], ' y: ', y_train[0])

### モデルの保存先の指定

In [ ]:
# モデルの保存先ディレクトリのpath
log_path = "./log/"
if os.path.exists(log_path):
    shutil.rmtree(log_path)
os.mkdir(log_path)

model_path = os.path.join(log_path, 'model.ckpt')

## 2.3 モデルの実装
- 入力部分では単語のidのベクトルを一度線形変換する.これは下の図のような行列演算で，単語のidが$i$の時は線形変換行列の$i$列目を取り出すのと等価である.<br>これは，batchでの処理はtf.nn.embedding_lookup(行列, idのリスト)によって行うことができる.<br>
<img src="figure/calc.png", width=600>

- 損失関数: Noise Contrastive Estimation (NCE) loss　<br>
出力単語の次元が膨大になっても現実的な時間内に計算が終わるように，近似によって計算量を抑える.<br>
Softmaxを計算すると計算量が単語の次元数$n$に依存して増えてしまい，計算量が$O(n)$になってしまう．<br>
ここで，代わりに負例となる単語$k$個をサンプリングし，ターゲットの周辺に現れる単語と負例となる単語を遠ざけるように学習する.<br>
これによって計算量は常に$O(k)$になる．

###  グラフの構築

In [ ]:
tf.reset_default_graph()

N_train = len(X_train)
# 単語の種類の数
vocab_size = len(word2id)

# パラメータ
# 学習率(learning rate)
lr = 1.0
# 学習回数
n_epoch = 8
# ミニバッチサイズ
batch_size = 128
# word2vecのベクトルの次元
embed_dim = 100
# 負例をサンプリングする数
num_sampled = 32

# 入力
x = tf.placeholder(tf.int32, shape=[None])
y = tf.placeholder(tf.int32, shape=[None, 1])

# 単語ごとの埋め込みベクトルの一覧行列．ランダムで初期化する．
embed_W = tf.Variable(
    tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0), name='word_embedding')

# 単語のidから埋め込みを取得する．
## TODO
hidden = tf.nn.embedding_lookup(embed_W, x)
## TODO

# 埋め込み行列から出力層のネットワークのパラメータ
nce_weights = tf.Variable(
    tf.truncated_normal([vocab_size, embed_dim],
                        stddev=1.0 / math.sqrt(embed_dim)))
nce_biases = tf.Variable(tf.zeros([vocab_size]))

# 損失関数: Noise Contrastive Estimation loss
loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
                   biases=nce_biases,
                   inputs=hidden,
                   labels=y,
                   num_sampled=num_sampled,
                   num_classes=vocab_size))

# SGD(Stochastic Gradient Descent : 確率的勾配降下法)で目的関数を最小化する
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

## 2.4 学習

In [ ]:
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epoch):
        print ('epoch %d | ' % epoch, end="")

        sum_loss = 0        
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)
                              
        for i in range(0, N_train, batch_size):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batch_size]]
            y_batch = y_train[perm[i:i+batch_size]].reshape(-1, 1)
        
            feed_dict = {x:X_batch, y:y_batch}
            _, loss_val = sess.run([optimizer, loss], feed_dict=feed_dict)
            sum_loss += loss_val * X_batch.shape[0]

        print('Train loss %.5f' %(sum_loss/ N_train))
        
    # 学習されたベクトルの値を取得する
    final_embed = embed_W.eval()
    
    # モデルの保存
    saver.save(sess, model_path)
    
    # 埋め込み空間の可視化用 
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embed_W.name
    embedding.metadata_path = 'metadata.tsv'
    summary_writer = tf.summary.FileWriter(log_path)
    projector.visualize_embeddings(summary_writer, config)

    sorted_dict = sorted(word2id.items(), key=lambda x: x[1])
    words = ["{}\n".format(x[0]) for x in sorted_dict]
    with open("log/metadata.tsv", "w") as f:
        f.writelines(words)

### tensorboardを確認してみよう
- tensorboard --logdir ./log/ (logのディレクトリを指定)
- ブラウザ上でnotebookのhttp://{サーバーのアドレス}:6006/ にアクセスする．
- サーバーのアドレスとは，現在用いているjupyter notebookのURLのうち，"ec2-....compute.amazonaws.com"で表される形式の文字列．

下の画面のように単語のベクトルを可視化したものを確認することができる

<img src="figure/tensorboard.png", width=600>

<a name='使用例'></a>
## 3. 使用例
## 類似語検索
word2vecによって類似語を検索してみよう．

In [ ]:
# ベクトルの正規化
norm_embed = final_embed/np.linalg.norm(final_embed, axis=1, keepdims=True)

In [ ]:
#　コサイン距離が近い単語上位top_n個取得する
def get_sim_word(query, top_n=6):
    query = query/np.linalg.norm(query)
    cos = 1- np.dot(query[np.newaxis,:], norm_embed.T)[0]
    sim = np.argsort(cos)
    return cos[sim[:top_n]], sim[:top_n]

In [ ]:
input_word = 'president'
result = final_embed[word2id[input_word]]
scores, indices = get_sim_word(result)
for i, index in enumerate(indices):
    if id2word[index]!=input_word:
        print(i, ' : ', id2word[index], '(distance : {0:.2})'.format(scores[i]))